# 🚀 High-Performance Hybrid QML (ResNet + Quantum Ensemble)

This notebook runs the **Optimized Architecture** designed to achieve >85% accuracy.

### Key Features:
1.  **Transfer Learning**: Uses `ResNet18` (pretrained on ImageNet) as the feature extractor.
2.  **Quantum Transfer Learning**: Freezes the ResNet backbone to train *only* the Quantum Ensemble.
3.  **Classical-Quantum Ensemble**: Averages predictions from the Quantum Circuit and a Classical Head.
4.  **Speed**: Uses `default.qubit` with vectorized batching (approx 2-3 mins/epoch).

> **Important:** Make sure you have selected **T4 GPU** runtime.  
> Go to **Runtime → Change runtime type → T4 GPU**

In [ ]:
# 1. Setup Environment & Mount Drive
import os
import shutil
from google.colab import drive
from getpass import getpass

# Mount Drive
drive.mount('/content/drive')

# Configuration
REPO_URL = "https://github.com/DreamX55/QML-for-Medical-Imaging.git"
PROJECT_KEY = "qml_medical_imaging" 
PROJECT_ROOT = f"/content/{PROJECT_KEY}"

# --- CUSTOMIZE THESE PATHS IF NEEDED ---
DRIVE_DATASET_PATH = '/content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset'
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/qml_medical_imaging_outputs'
# ---------------------------------------

In [ ]:
# 2. Verify GPU is Available
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    print('⚠️  No GPU detected! Go to Runtime → Change runtime type → T4 GPU')

!nvidia-smi

In [ ]:
# 3. Clone/Update Repository & Install Deps
if os.path.exists(PROJECT_ROOT):
    print(f"Repo exists at {PROJECT_ROOT}. Pulling latest changes...")
    %cd {PROJECT_ROOT}
    !git pull
else:
    print(f"Cloning repo to {PROJECT_ROOT}...")
    %cd /content
    !git clone {REPO_URL} {PROJECT_KEY}

%cd {PROJECT_ROOT}
!pip install -r requirements.txt
!pip install -q pennylane-lightning

In [ ]:
# 4. Link Data & Outputs
data_link = os.path.join(PROJECT_ROOT, 'data/brain_mri')
output_link = os.path.join(PROJECT_ROOT, 'outputs')

def force_symlink(src, dst):
    if os.path.exists(dst) or os.path.islink(dst):
        if os.path.isdir(dst) and not os.path.islink(dst):
            shutil.rmtree(dst)
        else:
            os.remove(dst)
    os.symlink(src, dst)
    print(f"Linked {src} -> {dst}")

os.makedirs(os.path.join(PROJECT_ROOT, 'data'), exist_ok=True)
force_symlink(DRIVE_DATASET_PATH, data_link)

if not os.path.exists(DRIVE_OUTPUT_PATH):
    os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)
force_symlink(DRIVE_OUTPUT_PATH, output_link)

In [ ]:
# 5. Run High-Performance Training
# Uses ResNet18 Backbone (Frozen) + Classical-Quantum Ensemble

!python main.py --mode train \
    --data_dir data/brain_mri \
    --project_name qml_high_perf \
    --model_type hybrid \
    --backbone resnet18 \
    --freeze_backbone \
    --epochs 15 \
    --batch_size 32 \
    --num_workers 2 \
    --quantum_device default.qubit \
    --device cuda \
    --output_dir outputs/high_perf_resnet